# [第8章: ニューラルネット](https://nlp100.github.io/ja/ch08.html)
第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

## [70. 単語ベクトルの和による特徴量](https://nlp100.github.io/ja/ch08.html#70-%E5%8D%98%E8%AA%9E%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E3%81%AE%E5%92%8C%E3%81%AB%E3%82%88%E3%82%8B%E7%89%B9%E5%BE%B4%E9%87%8F)
問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例$x_i$の特徴ベクトル$\boldsymbol{x}_i$を並べた行列$X$と，正解ラベルを並べた行列（ベクトル）$Y$を作成したい．

$$
X = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \dots \\ 
  \boldsymbol{x}_n \\ 
\end{pmatrix} \in \mathbb{R}^{n \times d},
Y = \begin{pmatrix} 
  y_1 \\ 
  y_2 \\ 
  \dots \\ 
  y_n \\ 
\end{pmatrix} \in \mathbb{N}^{n}
$$

ここで，$n$は学習データの事例数であり，$\boldsymbol{x}_i \in \mathbb{R}^d$と$y_i \in \mathbb{N}$はそれぞれ，$i \in \{1, \dots, n\}$番目の事例の特徴量ベクトルと正解ラベルを表す．
なお，今回は「ビジネス」「科学技術」「エンターテイメント」「健康」の4カテゴリ分類である．$\mathbb{N}_{<4}$で$4$未満の自然数（$0$を含む）を表すことにすれば，任意の事例の正解ラベル$y_i$は$y_i \in \mathbb{N}_{<4}$で表現できる．
以降では，ラベルの種類数を$L$で表す（今回の分類タスクでは$L=4$である）．

$i$番目の事例の特徴ベクトル$\boldsymbol{x}_i$は，次式で求める．

$$
\boldsymbol{x}_i = \frac{1}{T_i} \sum_{t=1}^{T_i} \mathrm{emb}(w_{i,t})
$$

ここで，$i$番目の事例は$T_i$個の（記事見出しの）単語列$(w_{i,1}, w_{i,2}, \dots, w_{i,T_i})$から構成され，$\mathrm{emb}(w) \in \mathbb{R}^d$は単語$w$に対応する単語ベクトル（次元数は$d$）である．すなわち，$i$番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものが$\boldsymbol{x}_i$である．今回は単語ベクトルとして，問題60でダウンロードしたものを用いればよい．$300$次元の単語ベクトルを用いたので，$d=300$である．

$i$番目の事例のラベル$y_i$は，次のように定義する．

$$
y_i = \begin{cases}
0 & (\mbox{記事}x_i\mbox{が「ビジネス」カテゴリの場合}) \\
1 & (\mbox{記事}x_i\mbox{が「科学技術」カテゴリの場合}) \\
2 & (\mbox{記事}x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
3 & (\mbox{記事}x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases}
$$

なお，カテゴリ名とラベルの番号が一対一で対応付いていれば，上式の通りの対応付けでなくてもよい．

以上の仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．
* 学習データの特徴量行列: $X_{\rm train} \in \mathbb{R}^{N_t \times d}$
* 学習データのラベルベクトル: $Y_{\rm train} \in \mathbb{N}^{N_t}$
* 検証データの特徴量行列: $X_{\rm valid} \in \mathbb{R}^{N_v \times d}$
* 検証データのラベルベクトル: $Y_{\rm valid} \in \mathbb{N}^{N_v}$
* 評価データの特徴量行列: $X_{\rm test} \in \mathbb{R}^{N_e \times d}$
* 評価データのラベルベクトル: $Y_{\rm test} \in \mathbb{N}^{N_e}$

なお，$N_t, N_v, N_e$はそれぞれ，学習データの事例数，検証データの事例数，評価データの事例数である．

### 注意
$y_i$の定義は上記ではなく、以下のように$\mathbb{R}^4$上のone-hotベクトルとして定義を変更する．

$$
y_i = \begin{cases}
\begin{pmatrix} 1 & 0 & 0 & 0 & \end{pmatrix}^\top & (\mbox{記事}x_i\mbox{が「ビジネス」カテゴリの場合}) \\
\begin{pmatrix} 0 & 1 & 0 & 0 & \end{pmatrix}^\top & (\mbox{記事}x_i\mbox{が「科学技術」カテゴリの場合}) \\
\begin{pmatrix} 0 & 0 & 1 & 0 & \end{pmatrix}^\top & (\mbox{記事}x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
\begin{pmatrix} 0 & 0 & 0 & 1 & \end{pmatrix}^\top & (\mbox{記事}x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases}
$$


In [1]:
!ls Input/GoogleNews-vectors-negative300.bin

Input/GoogleNews-vectors-negative300.bin


In [2]:
!ls Output/Chapter06/*.txt | grep -v feature.txt

Output/Chapter06/test.txt
Output/Chapter06/train.txt
Output/Chapter06/valid.txt


In [3]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np

In [4]:
%%time
wv = KeyedVectors.load_word2vec_format("Input/GoogleNews-vectors-negative300.bin", binary=True)
labels = ["b", "t", "e", "m"]

CPU times: user 20.9 s, sys: 5.92 s, total: 26.8 s
Wall time: 26.8 s


In [5]:
def save_features(file_type: str):
    if file_type != "train" and file_type != "valid" and file_type != "test":
        raise ValueError("file_type:%s" % file_type)

    in_path = "Output/Chapter06/%s.txt" % file_type
    out_paths = ("Output/Chapter08/x_%s.npy" % file_type,
                 "Output/Chapter08/y_%s.npy" % file_type)
    x, y, I = [], [], np.identity(len(labels))

    with open(in_path) as f:
        for line in f:
            elements = line.replace("\n", "").split("\t")
            label, words = elements[0], elements[1].split()
            wvs = [wv[word] if word in wv else np.zeros(300) for word in words]
            x.append(sum(wvs) / len(words))
            y.append(I[labels.index(label)])

    np.save(out_paths[0], np.array(x))
    np.save(out_paths[1], np.array(y))

In [6]:
%%time
save_features("train")
save_features("valid")
save_features("test")

CPU times: user 720 ms, sys: 56 ms, total: 776 ms
Wall time: 868 ms


## [71. 単層ニューラルネットワークによる予測](https://nlp100.github.io/ja/ch08.html#71-%E5%8D%98%E5%B1%A4%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AB%E3%82%88%E3%82%8B%E4%BA%88%E6%B8%AC)
問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

$$
\hat{\boldsymbol{y}}_1 = {\rm softmax}(\boldsymbol{x}_1 W), \\
\hat{Y} = {\rm softmax}(X_{[1:4]} W)
$$

ただし，${\rm softmax}$はソフトマックス関数，$X_{[1:4]} \in \mathbb{R}^{4 \times d}$は特徴ベクトル$\boldsymbol{x}_1, \boldsymbol{x}_2, \boldsymbol{x}_3, \boldsymbol{x}_4$を縦に並べた行列である．

$$
X_{[1:4]} = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \boldsymbol{x}_3 \\ 
  \boldsymbol{x}_4 \\ 
\end{pmatrix}
$$

行列$W \in \mathbb{R}^{d \times L}$は単層ニューラルネットワークの重み行列で，ここではランダムな値で初期化すればよい（問題73以降で学習して求める）．なお，$\hat{\boldsymbol{y}}_1 \in \mathbb{R}^L$は未学習の行列$W$で事例$x_1$を分類したときに，各カテゴリに属する確率を表すベクトルである．
同様に，$\hat{Y} \in \mathbb{R}^{n \times L}$は，学習データの事例$x_1, x_2, x_3, x_4$について，各カテゴリに属する確率を行列として表現している．

## [72. 損失と勾配の計算](https://nlp100.github.io/ja/ch08.html#72-%E6%90%8D%E5%A4%B1%E3%81%A8%E5%8B%BE%E9%85%8D%E3%81%AE%E8%A8%88%E7%AE%97)
学習データの事例$x_1$と事例集合$x_1, x_2, x_3, x_4$に対して，クロスエントロピー損失と，行列$W$に対する勾配を計算せよ．なお，ある事例$x_i$に対して損失は次式で計算される．

$$
l_i = - \log [\mbox{事例}x_i\mbox{が}y_i\mbox{に分類される確率}]
$$

ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．

## [73. 確率的勾配降下法による学習](https://nlp100.github.io/ja/ch08.html#73-%E7%A2%BA%E7%8E%87%E7%9A%84%E5%8B%BE%E9%85%8D%E9%99%8D%E4%B8%8B%E6%B3%95%E3%81%AB%E3%82%88%E3%82%8B%E5%AD%A6%E7%BF%92)
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列$W$を学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

## [74. 正解率の計測](https://nlp100.github.io/ja/ch08.html#74-%E6%AD%A3%E8%A7%A3%E7%8E%87%E3%81%AE%E8%A8%88%E6%B8%AC)
問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

## [75. 損失と正解率のプロット](https://nlp100.github.io/ja/ch08.html#75-%E6%90%8D%E5%A4%B1%E3%81%A8%E6%AD%A3%E8%A7%A3%E7%8E%87%E3%81%AE%E3%83%97%E3%83%AD%E3%83%83%E3%83%88)
問題73のコードを改変し，各エポックのパラメータ更新が完了するたびに，訓練データでの損失，正解率，検証データでの損失，正解率をグラフにプロットし，学習の進捗状況を確認できるようにせよ．

## [76. チェックポイント](https://nlp100.github.io/ja/ch08.html#76-%E3%83%81%E3%82%A7%E3%83%83%E3%82%AF%E3%83%9D%E3%82%A4%E3%83%B3%E3%83%88)
問題75のコードを改変し，各エポックのパラメータ更新が完了するたびに，チェックポイント（学習途中のパラメータ（重み行列など）の値や最適化アルゴリズムの内部状態）をファイルに書き出せ．

## [77. ミニバッチ化](https://nlp100.github.io/ja/ch08.html#77-%E3%83%9F%E3%83%8B%E3%83%90%E3%83%83%E3%83%81%E5%8C%96)
問題76のコードを改変し，$B$事例ごとに損失・勾配を計算し，行列$W$の値を更新せよ（ミニバッチ化）．$B$の値を$1, 2, 4, 8, \dots$と変化させながら，1エポックの学習に要する時間を比較せよ．

## [78. GPU上での学習](https://nlp100.github.io/ja/ch08.html#78-gpu%E4%B8%8A%E3%81%A7%E3%81%AE%E5%AD%A6%E7%BF%92)
問題77のコードを改変し，GPU上で学習を実行せよ．

## [79. 多層ニューラルネットワーク](https://nlp100.github.io/ja/ch08.html#79-%E5%A4%9A%E5%B1%A4%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF)
問題78のコードを改変し，バイアス項の導入や多層化など，ニューラルネットワークの形状を変更しながら，高性能なカテゴリ分類器を構築せよ．